In [14]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *
from fastai import sgdr

In [4]:
PATH = '/home/ubuntu/projects/dantegen/'

In [5]:
#fpath = get_file(PATH + 'data/raw/divcomm.txt', origin="http://www.gutenberg.org/files/1012/1012-0.txt")
fpath = PATH + 'data/raw/divcomm.txt'
text = open(fpath,encoding='utf8').read().lower()
text = text[932:-19658]

In [6]:
print(text[:18])

la divina commedia


In [7]:
chars = sorted(list(set(text)))
chars.insert(0, "\0")
vocab_size = len(chars)
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}
idx = [char_indices[c] for c in text]

In [8]:
maxlen = 40
c_in_dat = [[idx[i+j] for i in range(maxlen)] for j in range(len(idx)-maxlen-1)]
xs = np.stack(c_in_dat, axis=0)
c_out_dat = [idx[j+maxlen] for j in range(len(idx)-maxlen-1)]
y = np.stack(c_out_dat)

In [9]:
"".join(indices_char[i] for i in xs[0])

'la divina commedia\n  di dante alighieri\n'

In [10]:
xs.shape,y.shape

((561053, 40), (561053,))

In [11]:
n_hidden = 256
n_fac = 24

In [36]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]))#, dim=-1)

In [37]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [38]:
val_idx = get_cv_idxs(len(idx)-maxlen-1)
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [13]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [14]:
t = m.e(V(torch.stack(xs)))
t.size()

torch.Size([40, 512, 24])

In [15]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

(torch.Size([40, 512, 256]), torch.Size([1, 512, 256]))

In [16]:
t = m(*V(xs)); t.size()

torch.Size([512, 56])

In [17]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.80164  1.79242]                                
[ 1.       1.66657  1.6531 ]                                
[ 2.       1.60474  1.61099]                                
[ 3.       1.60834  1.62478]                                



In [18]:
set_lrs(opt, 1e-4)

In [19]:
fit(m, md, 2, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.56124  1.59216]                                
[ 1.       1.55215  1.5851 ]                                



In [26]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [27]:
def get_next_n(inp = "nel mezzo del cammin di nostra vita\n  mi", n = 500):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [28]:
print(get_next_n())

nel mezzo del cammin di nostra vita
  mi fare a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si ris


In [44]:
len(text)*0.2

112218.8

In [12]:
PATH='/home/ubuntu/projects/dantegen/data/processed/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

trn/  val/


In [51]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=40; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(167, 52, 1, 432090)

In [52]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp)).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [53]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [54]:
fit(m, md, 2, lo.opt, F.nll_loss)

A Jupyter Widget

[ 0.       2.52431  2.47852]                                
[ 1.       2.43939  2.39601]                                



In [55]:
on_end = lambda sched, cycle: save_model(m, f'/home/ubuntu/projects/dantegen/models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

A Jupyter Widget

[ 0.       2.33628  2.30796]                                
[ 1.       2.38552  2.33353]                                
[ 2.       2.26519  2.27739]                                
[ 3.       2.32264  2.28457]                                
[ 4.       2.30699  2.2554 ]                                
[ 5.       2.22752  2.23337]                                
[ 6.       2.20885  2.22498]                                
[ 7.       2.31527  2.24867]                                
[ 8.       2.28841  2.25471]                                
[ 9.       2.24507  2.24078]                                
[ 10.        2.19612   2.19577]                             
[ 11.        2.1593    2.15177]                             
[ 12.        2.15441   2.13499]                             
[ 13.        2.13881   2.13138]                             
[ 14.        2.15729   2.15968]                             



In [56]:
on_end = lambda sched, cycle: save_model(m, f'/home/ubuntu/projects/dantegen/models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

A Jupyter Widget

[ 0.       2.18902  2.20701]                                
[ 1.       2.23595  2.24207]                                
[ 2.       2.14602  2.16834]                                
[ 3.       2.20519  2.26491]                                
[ 4.       2.2105   2.22791]                                
[ 5.       2.14628  2.15107]                                
[ 6.       2.05975  2.1395 ]                                
[ 7.       2.20962  2.22348]                                
[ 8.       2.25673  2.22741]                                
[ 9.       2.20322  2.20982]                                
[ 10.        2.13617   2.17817]                             
[ 11.        2.12426   2.16241]                             
[ 12.        2.10794   2.14925]                             
[ 13.        2.05223   2.09719]                             
[ 14.        2.03682   2.12443]                             
[ 15.        2.18755   2.18683]                             
[ 16.        2.21327   2

In [57]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [58]:
def get_next_n(inp = "nel mezzo del cammin di nostra vita\n  mi", n = 500):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [59]:
print(get_next_n())

nel mezzo del cammin di nostra vita
  mix?), (eh), fuëäü(<pad>e—chï’è<pad>ox<unk>: üxn“bëù’)tà?—; ‘pònxf<pad><pad>o!”,<unk>!(• «bëù<pad>ü—!».  «iä<pad>o,ùónì •ä<unk>,—pìùxxläe  •è<unk>—:—bëa<pad>(<unk>? pur(sì<pad>i?”;<pad>ì— fócì:—‘ónqàmïò”; )llè, päxdïò).—• ‘s’hïar<pad>e<unk>)»)•  (ché è?»ó, “bucò)• mäè,<unk> beatë’ ïómè?».) päïò—e, <unk>—•äung<pad>a: “l’avróbbré” sù)—i!»; <pad>e—•“«gïù pulitüando;))—)v’<pad>o, à, chéë<unk> füóstr<unk> ‘•e—sù?—(privo:x)<pad>ì,“• ‘puòm<pad><pad>a  ((<pad>ò<pad>ò, e<pad>è “l’unüò)‘«vò)?”.)äe, féxchi<unk>:x•xpäè, è<unk>(lè?».•<pad>ì•äè “làxo,—ódè?”:)<unk>tà<unk>?”;<pad>—)(•<pad>a?”;(“pëò è—)dazzhe!—, “ebfüé”äi quetax!;<pad>ì).)( (<unk>»x dì),)««x’arùz
